In [1]:
library(mlflow)
library(glmnet)
library(carrier)

mlflow_set_tracking_uri("http://mlflow.bayescluster.com")

# Read the wine-quality csv file
data <- read.csv("https://raw.githubusercontent.com/databricks/mlflow-example-sklearn-elasticnet-wine/master/wine-quality.csv")

# Split the data into training and test sets. (0.75, 0.25) split.
sampled <- sample(1:nrow(data), 0.75 * nrow(data))
train <- data[sampled, ]
test <- data[-sampled, ]

# The predicted column is "quality" which is a scalar from [3, 9]
train_x <- as.matrix(train[, !(names(train) == "quality")])
test_x <- as.matrix(test[, !(names(train) == "quality")])
train_y <- train[, "quality"]
test_y <- test[, "quality"]

alpha <- mlflow_param("alpha", 0.5, "numeric")
lambda <- mlflow_param("lambda", 0.5, "numeric")

with(mlflow_start_run(), {
    model <- glmnet(train_x, train_y, alpha = alpha, lambda = lambda, family= "gaussian", standardize = FALSE)
    predictor <- crate(~ glmnet::predict.glmnet(!!model, as.matrix(.x)), !!model)
    predicted <- predictor(test_x)

    rmse <- sqrt(mean((predicted - test_y) ^ 2))
    mae <- mean(abs(predicted - test_y))
    r2 <- as.numeric(cor(predicted, test_y) ^ 2)

    message("Elasticnet model (alpha=", alpha, ", lambda=", lambda, "):")
    message("  RMSE: ", rmse)
    message("  MAE: ", mae)
    message("  R2: ", r2)

    mlflow_log_param("alpha", alpha)
    mlflow_log_param("lambda", lambda)
    mlflow_log_metric("rmse", rmse)
    mlflow_log_metric("r2", r2)
    mlflow_log_metric("mae", mae)

    mlflow_log_model(predictor, "model")
})

Loading required package: Matrix

Loaded glmnet 4.0

Elasticnet model (alpha=0.5, lambda=0.5):

  RMSE: 0.825732567270273

  MAE: 0.633544433728459

  R2: 0.155487906900828



ERROR: Error in get_python_bin(): MLflow not configured, please run install_mlflow() or set MLFLOW_PYTHON_BIN and MLFLOW_BIN environment variables.


In [ ]:
install.packages("carrier")